In [1]:
import torch
import numpy as np

In [2]:
# institate testing vectors
size = 10000
mask_size = int(np.round(size * size * 0.01))
x = torch.rand((size, 1))
y = torch.rand((1, size))

In [21]:
def compute_mask(w, topk_percentage):
    threshold = np.quantile(w.reshape(-1), topk_percentage)
    mask = np.where(w >= threshold)
    return mask

In [4]:
m = x.mm(y)
mask = compute_mask(m, 0.99)

In [19]:
def masked_outer_product(x, y, mask):
    values = torch.zeros(len(mask[0]))
    x = x.reshape(-1)
    y = y.reshape(-1)
    for i in range(len(mask[0])):
        values[i] = x[mask[0][i]] * y[mask[1][0]]
    
    return torch.sparse_coo_tensor(mask, values, (len(x), len(y)))
        

In [30]:
%timeit masked_outer_product(x, y, mask)

8.89 s ± 97.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%timeit m = x.mm(y)
# torch.sparse_coo_tensor(mask, m[mask], m.shape)

132 ms ± 2.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Benchmarkig torch.sparse vs elementwise masking

In [24]:
size = 1000
mat1 = torch.rand((size, size))
vec_mask = compute_mask(mat1, 0.99)
vector = torch.rand((size, 1))

In [25]:
len(vec_mask[0])

10001

In [27]:
%timeit mat1_mask = torch.zeros_like(mat1)
%timeit mat1_mask[vec_mask] = 1
%timeit (mat1 * mat1_mask).mm(vector)

1.01 ms ± 21.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
103 µs ± 572 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.38 ms ± 43.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [28]:
%timeit sparse_mat1 = torch.sparse_coo_tensor(vec_mask, mat1[vec_mask], mat1.shape)
%timeit sparse_mat1.mm(vector)

2.56 ms ± 108 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
17.6 ms ± 226 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
